In [1]:
import pandas as pd


In [2]:
df = pd.read_csv('data.csv')
df.head()

,author,poem,language,name,tags
0,Владимир Соловьев,Дождались меня белые ночи\nНад простором густы...,RU,Владимир Соловьев,[]
1,Сергей Есенин,"В этот лес завороженный,\nПо пушинкам серебра,...",RU,Что это такое?,"['Для детей', 'О природе', 'Серебряный век', '..."
2,Александр Пушкин,У лукоморья дуб зелёный;\nЗлатая цепь на дубе ...,RU,У лукоморья дуб зелёный (отрывок из поэмы «Рус...,"['Для детей', 'Золотой век', 'Стихи Александра..."
3,Михаил Лермонтов,"— Скажи-ка, дядя, ведь недаром\nМосква, спален...",RU,Бородино,"['О войне', 'О родине', 'Для детей', 'Длинные'..."
4,Александр Пушкин,Я к вам пишу — чего же боле?\nЧто я могу еще с...,RU,Письмо Татьяны к Онегину (отрывок из романа «Е...,"['О любви', 'Золотой век', 'Стихи Александра П..."


## Data Cleaning
For now, we need to get rid of column language, because dataset was scraped only from russian website. 
Also, there we need to split the words in the poem and remove the tags that contain authors name, because they are irrelevant to our analysis.

In [3]:
# Let's start with the tags
import numpy as np
np_arr = df.to_numpy()

print(np_arr[1])

['Сергей Есенин'
 'В этот лес завороженный,\nПо пушинкам серебра,\nЯ с винтовкой заряженной\nНа охоту шел вчера.\nПо дорожке чистой, гладкой\nЯ прошел, не наследил…\nКто ж катался здесь украдкой?\nКто здесь падал и ходил?\nПодойду, взгляну поближе:\nХрупкий снег изломан весь.\nЗдесь вот когти, дальше — лыжи…\nКто-то странный бегал здесь.\nКабы твердо знал я тайну\nЗаколдованным речам,\nЯ узнал бы хоть случайно,\nКто здесь бродит по ночам.\nИз-за елки бы высокой\nПодсмотрел я на кругу:\nКто глубокий след далекий\nОставляет на снегу?..\n'
 'RU' 'Что это такое?'
 "['Для детей', 'О природе', 'Серебряный век', 'Стихи Сергея Есенина – Для детей', 'Стихи Сергея Есенина – О природе', 'Стихи Сергея Есенина – Серебряный век']"]


In [4]:
# convert the last column from string to array of strings

for record in np_arr:
    to_convert = record[4]
    if to_convert != "[]":
        tags_arr = to_convert.translate({
        ord("[") : None,
        ord(","): None,
        ord("]"): None
        }).split("' '")
        tags_arr[0]=tags_arr[0].replace("'", "")
        tags_arr[-1]=tags_arr[-1].replace("'", "")
        record[4] = np.asarray(tags_arr)
    else:
        record[4] = np.nan

print(np_arr[0])
print(np_arr[1])



['Владимир Соловьев'
 'Дождались меня белые ночи\nНад простором густых островов…\nСнова смотрят знакомые очи,\nИ мелькает былое без слов.\nВ царство времени всё я не верю,\nСилу сердца еще берегу,\nРоковую не скрою потерю,\nНо сказать «навсегда» — не могу.\nПри мерцании долгом заката,\nПред минутной дремотою дня,\nЧто погиб его свет без возврата,\nВ эту ночь не уверишь меня.\n'
 'RU' 'Владимир Соловьев' nan]
['Сергей Есенин'
 'В этот лес завороженный,\nПо пушинкам серебра,\nЯ с винтовкой заряженной\nНа охоту шел вчера.\nПо дорожке чистой, гладкой\nЯ прошел, не наследил…\nКто ж катался здесь украдкой?\nКто здесь падал и ходил?\nПодойду, взгляну поближе:\nХрупкий снег изломан весь.\nЗдесь вот когти, дальше — лыжи…\nКто-то странный бегал здесь.\nКабы твердо знал я тайну\nЗаколдованным речам,\nЯ узнал бы хоть случайно,\nКто здесь бродит по ночам.\nИз-за елки бы высокой\nПодсмотрел я на кругу:\nКто глубокий след далекий\nОставляет на снегу?..\n'
 'RU' 'Что это такое?'
 array(['Для детей', '

In [5]:
# Now remove tags with author's name because they duplicate tags without
# For example
# Normal tag: About love
# Author tag: Sergey Yesenin Poems - About love
# Let's delete by the first letters of surname because it will be easier for us
for record in np_arr:
    if record[4] is not np.nan:
        author = record[0].split(" ")[0]
        author = author[:len(author)-3]
        indexes_to_delete = []
        for i in range(len(record[4])):
            if author in record[4][i]:
                indexes_to_delete.append(i)

        record[4] = np.delete(record[4], indexes_to_delete)

print(np_arr[2])

['Александр Пушкин'
 'У лукоморья дуб зелёный;\nЗлатая цепь на дубе том:\nИ днём и ночью кот учёный\nВсё ходит по цепи кругом;\nИдёт направо — песнь заводит,\nНалево — сказку говорит.\nТам чудеса: там леший бродит,\nРусалка на ветвях сидит;\nТам на неведомых дорожках\nСледы невиданных зверей;\nИзбушка там на курьих ножках\nСтоит без окон, без дверей;\nТам лес и дол видений полны;\nТам о заре прихлынут волны\nНа брег песчаный и пустой,\nИ тридцать витязей прекрасных\nЧредой из вод выходят ясных,\nИ с ними дядька их морской;\nТам королевич мимоходом\nПленяет грозного царя;\nТам в облаках перед народом\nЧерез леса, через моря\nКолдун несёт богатыря;\nВ темнице там царевна тужит,\nА бурый волк ей верно служит;\nТам ступа с Бабою Ягой\nИдёт, бредёт сама собой,\nТам царь Кащей над златом чахнет;\nТам русский дух… там Русью пахнет!\nИ там я был, и мёд я пил;\nУ моря видел дуб зелёный;\nПод ним сидел, и кот учёный\nСвои мне сказки говорил.\nЧитать полное произведение.\n'
 'RU' 'У лукоморья дуб

In [6]:
#Tokenize words
import nltk
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation

nltk.download("stopwords")
mystem = Mystem() 
russian_stopwords = stopwords.words("russian")

def preprocess_text(text):
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords\
              and token != " " \
              and token != "—" \
              and token != "…" \
              and token != "?.."
              and token.strip() not in punctuation]
    
    text = " ".join(tokens)
    
    return text



[nltk_data] Downloading package stopwords to
[nltk_data]     /home/kinatra/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
df = pd.DataFrame(np_arr, columns=df.columns)
df.tail()

,author,poem,language,name,tags
1182,Булат Окуджава,"Когда мне невмочь пересилить беду,\nкогда подс...",RU,Полночный троллейбус,"[О жизни, О Москве, Советские]"
1183,Алексей Кольцов,"Я был у ней; она сказала:\n«Люблю тебя, мой ми...",RU,Алексей Кольцов,"[О любви, Золотой век]"
1184,Алексей Фатьянов,"Когда весна придёт, не знаю.\nПридут дожди… Со...",RU,Алексей Фатьянов,"[О жизни, О родине, Советские, Песни]"
1185,Евгений Баратынский,"Шуми, шуми с крутой вершины,\nНе умолкай, пото...",RU,Евгений Баратынский,[О природе]
1186,Роберт Рождественский,Сорок трудный год.\nОмский госпиталь…\nКоридор...,RU,Концерт,"[О войне, Советские]"


In [8]:
df["poem"] = df["poem"].apply(preprocess_text)
df.head()

,author,poem,language,name,tags
0,Владимир Соловьев,дождаться белый ночь простор густой остров сно...,RU,Владимир Соловьев,NaN
1,Сергей Есенин,лес завораживать пушинка серебро винтовка заря...,RU,Что это такое?,"[Для детей, О природе]"
2,Александр Пушкин,лукоморье дуб зеленый златой цепь дуб день ноч...,RU,У лукоморья дуб зелёный (отрывок из поэмы «Рус...,"[Для детей, Золотой век]"
3,Михаил Лермонтов,— сказать ка дядя недаром москва спалить пожа...,RU,Бородино,"[О войне, О родине, Для детей, Длинные, О Моск..."
4,Александр Пушкин,писать — боле мочь сказать знать ваш воля пр...,RU,Письмо Татьяны к Онегину (отрывок из романа «Е...,"[О любви, Золотой век]"


In [9]:
import re
df['poem']=df['poem'].apply(lambda x: re.sub('\w*\d\w*','', x))

In [10]:
# Finally drop a language column
df = df.drop(columns=['language'])

In [11]:
df["poem"].head()

0    дождаться белый ночь простор густой остров сно...
1    лес завораживать пушинка серебро винтовка заря...
2    лукоморье дуб зеленый златой цепь дуб день ноч...
3    —  сказать ка дядя недаром москва спалить пожа...
4    писать  —  боле мочь сказать знать ваш воля пр...
Name: poem, dtype: object

In [12]:
df.isnull().sum()

author     0
poem       0
name       0
tags      39
dtype: int64

In [13]:
# We can get rid of poems with no tags, they are 39 out of 1186
df = df.dropna(axis=0)

df.isnull().sum()

author    0
poem      0
name      0
tags      0
dtype: int64

In [14]:
df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1148 entries, 1 to 1186
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   author  1148 non-null   object
 1   poem    1148 non-null   object
 2   name    1148 non-null   object
 3   tags    1148 non-null   object
dtypes: object(4)
memory usage: 44.8+ KB


In [15]:
# Group by authors
df_grouped=df[['author','poem']].groupby(by='author').agg(lambda x:' '.join(x))
df_grouped.head()

,poem
author,
Агния Барто,решать иван меняться обман менять —\n это сут...
Александр Башлачев,долго идти зной мороз весь снести оставаться в...
Александр Блок,ночь улица фонарь аптека бессмысленный тусклый...
Александр Введенский,жалко зверь бегать синий дорожка говорить пове...
Александр Вертинский,холоднойгде целовать палец уходить ваш китайчо...


In [16]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(analyzer='word')

data=cv.fit_transform(df_grouped['poem'])
df_dtm = pd.DataFrame(data.toarray(), columns=cv.get_feature_names())
df_dtm.index=df_grouped.index
df_dtm.head()

/home/kinatra/.local/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,___,______,_______,________,_________,_______________,___________________,aberwrach,allo,auf,...,ятвяг,ять,яхонт,яхта,ячейка,яшма,ящерица,ящик,ящичек,яяяяя
author,,,,,,,,,,,,,,,,,,,,,
Агния Барто,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Александр Башлачев,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Александр Блок,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
Александр Введенский,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Александр Вертинский,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
from wordcloud import WordCloud
from textwrap import wrap
import matplotlib.pyplot as plt

def generate_wordcloud(data,title, option):
  wc = WordCloud(width=400, height=330, max_words=150,colormap="Dark2").generate_from_frequencies(data)
  if option == "save":
    wc.to_file(f'poets_wordcloud/{title}.png')
  else:
    plt.figure(figsize=(10,8))
    plt.imshow(wc, interpolation='bilinear')
    plt.axis("off")
    plt.title('\n'.join(wrap(title,60)),fontsize=13)
    plt.show()
  
# Transposing document term matrix
df_dtm=df_dtm.transpose()

# Plotting word cloud for each author
for index,poem in enumerate(df_dtm.columns):
  generate_wordcloud(df_dtm[poem].sort_values(ascending=False),poem, "save")


## Conclusion
We've collected poetry data by using Web Scrapping. Next, data was cleaned and analysed. Wordcloud was created and we can see most common words author has used in his poems. To futher analysis - which words are most common for each tag.

